In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

cudnn.benchmark = True #-- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
                       #-- If this is set to false, uses some in-built heuristics that might not always be fastest.

cudnn.fastest = True #-- this is like the :fastest() mode for the Convolution modules,
                     #-- simply picks the fastest convolution algorithm, rather than tuning for workspace size

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim

import random, os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from models.resnet import *
from utils import *

In [3]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config.ini")
# path to the dataset
path_to_FOURVIERE_CLEAN2 = Config.get("Corpus", 'path_to_FOURVIERE_CLEAN2')

In [4]:
imSize = 225
batchSize = 32
nb_epoch = 50

In [5]:
# Load train and test data

pathToImg = path_to_FOURVIERE_CLEAN2+"IMAGE_FROM_V/"
# Dictionary with the imgName as key and corresponding class as value for the test and train part
ImgNameToClassTrain = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()}
ImgNameToClassTest = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()}
# Convert the class to a numeric id
ClassToIdClass = {Artworks:i for i, Artworks in enumerate(sorted(set([line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()])))}
# Associate imgName to the corresponding numeric class id
ImgNameToIdClassTrain = {ImgName:ClassToIdClass[ImgNameToClassTrain[ImgName]] for ImgName in  ImgNameToClassTrain.keys()}
ImgNameToIdClassTest = {ImgName:ClassToIdClass[ImgNameToClassTest[ImgName]] for ImgName in  ImgNameToClassTest.keys()}

# open all images
dImgTrain = {imgName:Image.open(pathToImg+imgName) for imgName in ImgNameToClassTrain.keys()}
dImgTest = {imgName:Image.open(pathToImg+imgName).resize((imSize, imSize), Image.BILINEAR) for imgName in ImgNameToClassTest.keys()}

In [6]:
# Compute mean and standard deviation for images normalisation
mean = ComputeMean(dImgTrain.values())
std = ComputeStdDev(dImgTrain.values())

imageTrainTransform = transforms.Compose([
    transforms.Scale(300), 
    transforms.RandomCrop(225), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

imageTestTransform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

In [7]:
criterion = nn.CrossEntropyLoss()

In [7]:
# defined a new net with the number of classes corresponding to the dataset
ResnetFineTuned = resnet50(len(ImgNameToIdClassTrain)).train()
# load the pre-trained model and copy only the Features 
copyFeaturesParametersResnet(ResnetFineTuned, resnet50(pretrained=True), 3, 4, 6, 3, "Bottleneck")
ResnetFineTuned = ResnetFineTuned.cuda()

copy net.conv1 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
copy net.bn1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
copy layer1 Sequential (
  (0): Bottleneck (
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU (inplace)
    (downsample): Sequential (
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    )
  )
  (1): Bottleneck (
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, af

In [8]:
#define the optimizer to train only the fully connected layer lr = 0.01
optimizer=optim.SGD([{'params': ResnetFineTuned.fc.parameters()},
                     {'params': ResnetFineTuned.conv1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.bn1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer2.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer3.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer4.parameters(), 'lr': 0.0},
                    ], lr=0.01, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

epoch 0 test : #Correct 44 on 624 (7.1%)
epoch 1 test : #Correct 85 on 624 (13.6%)
epoch 2 test : #Correct 151 on 624 (24.2%)
epoch 3 test : #Correct 190 on 624 (30.4%)
epoch 4 test : #Correct 212 on 624 (34.0%)
epoch 5 test : #Correct 214 on 624 (34.3%)
epoch 6 test : #Correct 243 on 624 (38.9%)
epoch 7 test : #Correct 257 on 624 (41.2%)
epoch 8 test : #Correct 253 on 624 (40.5%)
epoch 9 test : #Correct 293 on 624 (47.0%)
epoch 10 test : #Correct 271 on 624 (43.4%)
epoch 11 test : #Correct 299 on 624 (47.9%)
epoch 12 test : #Correct 281 on 624 (45.0%)
epoch 13 test : #Correct 295 on 624 (47.3%)
epoch 14 test : #Correct 305 on 624 (48.9%)
epoch 15 test : #Correct 296 on 624 (47.4%)
epoch 16 test : #Correct 305 on 624 (48.9%)
epoch 17 test : #Correct 315 on 624 (50.5%)
epoch 18 test : #Correct 310 on 624 (49.7%)
epoch 19 test : #Correct 313 on 624 (50.2%)
epoch 20 test : #Correct 326 on 624 (52.2%)
epoch 21 test : #Correct 318 on 624 (51.0%)
epoch 22 test : #Correct 309 on 624 (49.5%)
e

In [9]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01"".ckpt")

In [10]:
#define the optimizer to train only the fully connected layer lr = 0.001
optimizer=optim.SGD([{'params': ResnetFineTuned.fc.parameters()},
                     {'params': ResnetFineTuned.conv1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.bn1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer1.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer2.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer3.parameters(), 'lr': 0.0},
                     {'params': ResnetFineTuned.layer4.parameters(), 'lr': 0.0},
                    ], lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

epoch 0 test : #Correct 328 on 624 (52.6%)
epoch 1 test : #Correct 336 on 624 (53.8%)
epoch 2 test : #Correct 335 on 624 (53.7%)
epoch 3 test : #Correct 331 on 624 (53.0%)
epoch 4 test : #Correct 337 on 624 (54.0%)
epoch 5 test : #Correct 328 on 624 (52.6%)
epoch 6 test : #Correct 331 on 624 (53.0%)
epoch 7 test : #Correct 326 on 624 (52.2%)
epoch 8 test : #Correct 324 on 624 (51.9%)
epoch 9 test : #Correct 335 on 624 (53.7%)
epoch 10 test : #Correct 327 on 624 (52.4%)
epoch 11 test : #Correct 331 on 624 (53.0%)
epoch 12 test : #Correct 332 on 624 (53.2%)
epoch 13 test : #Correct 322 on 624 (51.6%)
epoch 14 test : #Correct 322 on 624 (51.6%)
epoch 15 test : #Correct 318 on 624 (51.0%)
epoch 16 test : #Correct 326 on 624 (52.2%)
epoch 17 test : #Correct 330 on 624 (52.9%)
epoch 18 test : #Correct 323 on 624 (51.8%)
epoch 19 test : #Correct 328 on 624 (52.6%)
epoch 20 test : #Correct 330 on 624 (52.9%)
epoch 21 test : #Correct 327 on 624 (52.4%)
epoch 22 test : #Correct 331 on 624 (53.0%

In [11]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")

In [12]:
ResnetFineTuned = torch.load("ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")
ResnetFineTuned = ResnetFineTuned.cuda()

In [13]:
# load the model and define the optimizer to train net with lr of 0.001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

epoch 0 test : #Correct 329 on 624 (52.7%)
epoch 1 test : #Correct 338 on 624 (54.2%)
epoch 2 test : #Correct 346 on 624 (55.4%)
epoch 3 test : #Correct 324 on 624 (51.9%)
epoch 4 test : #Correct 336 on 624 (53.8%)
epoch 5 test : #Correct 307 on 624 (49.2%)
epoch 6 test : #Correct 319 on 624 (51.1%)
epoch 7 test : #Correct 338 on 624 (54.2%)
epoch 8 test : #Correct 328 on 624 (52.6%)
epoch 9 test : #Correct 336 on 624 (53.8%)
epoch 10 test : #Correct 325 on 624 (52.1%)
epoch 11 test : #Correct 326 on 624 (52.2%)
epoch 12 test : #Correct 330 on 624 (52.9%)
epoch 13 test : #Correct 338 on 624 (54.2%)
epoch 14 test : #Correct 334 on 624 (53.5%)
epoch 15 test : #Correct 340 on 624 (54.5%)
epoch 16 test : #Correct 340 on 624 (54.5%)
epoch 17 test : #Correct 336 on 624 (53.8%)
epoch 18 test : #Correct 346 on 624 (55.4%)
epoch 19 test : #Correct 349 on 624 (55.9%)
epoch 20 test : #Correct 345 on 624 (55.3%)
epoch 21 test : #Correct 343 on 624 (55.0%)
epoch 22 test : #Correct 352 on 624 (56.4%

In [14]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001.ckpt")

In [15]:
# load the model and define the optimizer to train net with lr of 0.0001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.0001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)    

epoch 0 test : #Correct 352 on 624 (56.4%)
epoch 1 test : #Correct 349 on 624 (55.9%)
epoch 2 test : #Correct 355 on 624 (56.9%)
epoch 3 test : #Correct 348 on 624 (55.8%)
epoch 4 test : #Correct 356 on 624 (57.1%)
epoch 5 test : #Correct 355 on 624 (56.9%)
epoch 6 test : #Correct 354 on 624 (56.7%)
epoch 7 test : #Correct 353 on 624 (56.6%)
epoch 8 test : #Correct 358 on 624 (57.4%)
epoch 9 test : #Correct 363 on 624 (58.2%)
epoch 10 test : #Correct 357 on 624 (57.2%)
epoch 11 test : #Correct 356 on 624 (57.1%)
epoch 12 test : #Correct 355 on 624 (56.9%)
epoch 13 test : #Correct 356 on 624 (57.1%)
epoch 14 test : #Correct 350 on 624 (56.1%)
epoch 15 test : #Correct 357 on 624 (57.2%)
epoch 16 test : #Correct 351 on 624 (56.3%)
epoch 17 test : #Correct 359 on 624 (57.5%)
epoch 18 test : #Correct 358 on 624 (57.4%)
epoch 19 test : #Correct 353 on 624 (56.6%)
epoch 20 test : #Correct 353 on 624 (56.6%)
epoch 21 test : #Correct 357 on 624 (57.2%)
epoch 22 test : #Correct 362 on 624 (58.0%

In [16]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001-50epochAll-lr-0.0001.ckpt")

In [9]:
ResnetFineTuned = torch.load("ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001-50epochAll-lr-0.0001.ckpt")
ResnetFineTuned = ResnetFineTuned.cuda()

In [10]:
# load the model and define the optimizer to train net with lr of 0.0001
optimizer=optim.SGD(ResnetFineTuned.parameters(), lr=0.00001, momentum=0.9)
for epoch in range(50): # loop over the dataset multiple times
    print "epoch", epoch, 
    trainclassifier(ResnetFineTuned, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClassTrain, 1)
    testClassifier(ResnetFineTuned, dImgTest, imSize, imageTestTransform, ImgNameToIdClassTest, 1)   

epoch 0 test : #Correct 357 on 624 (57.2%)
epoch 1 test : #Correct 355 on 624 (56.9%)
epoch 2 test : #Correct 362 on 624 (58.0%)
epoch 3 test : #Correct 360 on 624 (57.7%)
epoch 4 test : #Correct 361 on 624 (57.9%)
epoch 5 test : #Correct 360 on 624 (57.7%)
epoch 6 test : #Correct 356 on 624 (57.1%)
epoch 7 test : #Correct 361 on 624 (57.9%)
epoch 8 test : #Correct 362 on 624 (58.0%)
epoch 9 test : #Correct 359 on 624 (57.5%)
epoch 10 test : #Correct 359 on 624 (57.5%)
epoch 11 test : #Correct 364 on 624 (58.3%)
epoch 12 test : #Correct 358 on 624 (57.4%)
epoch 13 test : #Correct 366 on 624 (58.7%)
epoch 14 test : #Correct 361 on 624 (57.9%)
epoch 15 test : #Correct 355 on 624 (56.9%)
epoch 16 test : #Correct 362 on 624 (58.0%)
epoch 17 test : #Correct 359 on 624 (57.5%)
epoch 18 test : #Correct 363 on 624 (58.2%)
epoch 19 test : #Correct 365 on 624 (58.5%)
epoch 20 test : #Correct 357 on 624 (57.2%)
epoch 21 test : #Correct 357 on 624 (57.2%)
epoch 22 test : #Correct 360 on 624 (57.7%

In [11]:
torch.save(ResnetFineTuned, "ckpt/ResnetFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epochAll-lr-0.001-50epochAll-lr-0.0001-50epochAll-lr-0.00001.ckpt")